A ideia foi importar 3 arquivos CSV de uma bucket da GCP, convertê-los em dicionário e enviá-los para uma coleção no mongoDB.
Alterei os nomes dos caminhos dos arquivos por questões de privacidade.

# Instalando e importando as bibliotecas necessárias

In [ ]:
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.cloud import storage
import os
import pandas as pd
import pymongo
from pymongo import MongoClient

# Lendo as bases de dados originais na Bucket

In [ ]:
#KEY do Service Account da GCP
serviceAccount = '/caminho_do_arquivo_do_certificado.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
#Configurações Google Cloud Storage
client = storage.Client()
bucket = client.get_bucket('nome da bucket na GCP')
bucket.blob('D.SDA.PDA.005.CAT.202201.csv')
path = 'gs://caminho_do_arquivo_na_gcp/D.SDA.PDA.005.CAT.202201.csv'

In [ ]:
# Lendo o arquivo CSV a partir da bucket
df1 = pd.read_csv('gs://caminho_do_arquivo_na_gcp/D.SDA.PDA.005.CAT.202201.csv', sep = ';', encoding = 'ISO-8859-1')

In [ ]:
client = storage.Client()
bucket = client.get_bucket('nome da bucket na GCP')
bucket.blob('D.SDA.PDA.005.CAT.202204.csv')
path = 'gs://caminho_do_arquivo_na_gcp/D.SDA.PDA.005.CAT.202204.csv'

In [ ]:
df2 = pd.read_csv('gs://caminho_do_arquivo_na_gcp/D.SDA.PDA.005.CAT.202204.csv', sep = ';', encoding = 'ISO-8859-1')

In [ ]:
client = storage.Client()
bucket = client.get_bucket('nome da bucket na GCP')
bucket.blob('D.SDA.PDA.005.CAT.202210.csv')
path = 'gs://caminho_do_arquivo_na_gcp/D.SDA.PDA.005.CAT.202210.csv'

In [ ]:
df3 = pd.read_csv('gs://caminho_do_arquivo_na_gcp/D.SDA.PDA.005.CAT.202210.csv', sep = ';', encoding = 'ISO-8859-1')

# Enviado as bases de dados originais para o MongoDB

In [ ]:
# Conector do Mongo Atlas
uri = "mongodb+srv://caminho_do_conector_mongoDB"
client = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile='/caminho_do_arquivo_do_cerificado.pem')

In [ ]:
# Conectando ao database e à coleção de arquivos originais e verificando a quantidade de documentos na coleção
db = client['nome_database']
collection = db['nome_colecao']
doc_count = collection.count_documents({})
print(doc_count)

0


Como os dfs 1 e 2 são muito grandes, há a necessidade de dividí-los para que seja possível realizar a importação para o mongoDB

In [ ]:
# Lendo a quantidade de linhas do df1
len(df1.axes[0])

152140

In [ ]:
# Lendo a quantidade de linhas do df2
len(df2.axes[0])

89602

In [ ]:
# Lendo a quantidade de linhas do df3
len(df3.axes[0])

15659

In [ ]:
# Dividindo o df1 em 4 subdfs
subdf1_1 = df1.iloc[:38035]
subdf1_2 = df1.iloc[38035:76070]
subdf1_3 = df1.iloc[76070:114105]
subdf1_4 = df1.iloc[114105:]

In [ ]:
# Dividindo o df2 em 3 subdfs
subdf2_1 = df2.iloc[:29867]
subdf2_2 = df2.iloc[29867:59734]
subdf2_3 = df2.iloc[59734:]

In [ ]:
# Transformando os dfs em dicionários
subdf1_1_dict = subdf1_1.to_dict('records')
subdf1_2_dict = subdf1_2.to_dict('records')
subdf1_3_dict = subdf1_3.to_dict('records')
subdf1_4_dict = subdf1_4.to_dict('records')
subdf2_1_dict = subdf2_1.to_dict('records')
subdf2_2_dict = subdf2_2.to_dict('records')
subdf2_3_dict = subdf2_3.to_dict('records')
df3_dict = df3.to_dict('records')

In [ ]:
# Enviando os dfs para a coleção arquivos_originais no MongoDB
collection.insert_many(subdf1_1_dict)
collection.insert_many(subdf1_2_dict)
collection.insert_many(subdf1_3_dict)
collection.insert_many(subdf1_4_dict)
collection.insert_many(subdf2_1_dict)
collection.insert_many(subdf2_2_dict)
collection.insert_many(subdf2_3_dict)
collection.insert_many(df3_dict)

In [ ]:
# Verificando a quantidade de documentos no MongoDB
doc_count = collection.count_documents({})
print(doc_count)

257401


Foi verificado que o total de documentos no mongoDB coincide com a quantidade total de linhas dos 3 dfs importados.